In [1]:
## Part 1 - Import a Dataset

import folium
import requests
import pandas as pd

# Read and extract the dataset and create a data table
crash_table = pd.read_csv("https://cmsc320.github.io/files/Baltimore_City_Vehicle_Crashes.csv")
crash_table

/tmp/ipykernel_714/3737045742.py:8: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  crash_table = pd.read_csv("https://cmsc320.github.io/files/Baltimore_City_Vehicle_Crashes.csv")


,YEAR,QUARTER,LIGHT_DESC,LIGHT_CODE,COUNTY_NO,MUNI_DESC,MUNI_CODE,JUNCTION_DESC,JUNCTION_CODE,COLLISION_TYPE_DESC,...,FEET_MILES_FLAG_DESC,FEET_MILES_FLAG,DISTANCE_DIR_FLAG,REFERENCE_NO,REFERENCE_TYPE_CODE,REFERENCE_SUFFIX,REFERENCE_ROAD_NAME,LATITUDE,LONGITUDE,LOCATION
0,2020,Q2,NaN,6.02,24.0,NaN,NaN,Non Intersection,1.0,Other,...,Miles,M,N,NaN,NaN,NaN,NORTH AVE,39.311025,-76.616429,POINT (-76.616429453205 39.311024794431)
1,2017,Q2,Daylight,1.00,24.0,NaN,NaN,NaN,NaN,Single Vehicle,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.282928,-76.635215,POINT (-76.6352150952347 39.2829284750108)
2,2020,Q2,Daylight,1.00,24.0,NaN,NaN,Intersection,2.0,Other,...,Feet,F,S,NaN,NaN,NaN,WINDSOR AVE,39.312903,-76.651472,POINT (-76.651471912939 39.312903404529)
3,2020,Q2,Daylight,1.00,24.0,NaN,NaN,NaN,99.0,Same Movement Angle,...,NaN,U,E,NaN,NaN,NaN,WASHINGTON ST,39.294944,-76.599329,POINT (-76.599328693204 39.294943770185)
4,2020,Q2,NaN,5.02,24.0,NaN,NaN,NaN,NaN,Same Direction Rear End,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.296874,-76.595871,POINT (-76.595871121891 39.296873988072)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135585,2021,Q4,Unknown,99.00,24.0,NaN,NaN,Non Intersection,1.0,Same Direction Sideswipe,...,Feet,F,E,NaN,NaN,NaN,LINDEN AVE.,39.315169,-76.635210,POINT (-76.635209753929 39.315168933901)
135586,2021,Q4,Daylight,1.00,24.0,NaN,NaN,Intersection,2.0,Single Vehicle,...,Feet,F,N,NaN,NaN,NaN,N CALVERT ST,39.291235,-76.612507,POINT (-76.61250671377 39.291234751225)
135587,2021,Q4,Dark Lights On,3.00,24.0,NaN,NaN,Intersection,2.0,Same Direction Rear End,...,Feet,F,S,NaN,NaN,NaN,E FAYETT,39.290660,-76.606670,POINT (-76.60667 39.29066)
135588,2021,Q4,Dark Lights On,3.00,24.0,NaN,999.0,Non Intersection,1.0,Same Direction Rear End Left Turn,...,Feet,F,W,2125.0,MU,NaN,GARRISON BLVD,39.343406,-76.640647,POINT (-76.640646523492 39.343405723453)


In [2]:
## Part 2 - Creating a Map

# Create a map of Baltimore
map_osm = folium.Map(location=[39.29, -76.61], zoom_start=13)
map_osm

In [3]:
## Part 3 - Create Markers

from numpy.random import default_rng

# Create a subset table to extract only the necessary columns from the original table
subset_table = crash_table[["YEAR", "SURF_COND_DESC", "LATITUDE", "LONGITUDE", "LOCATION"]].dropna()
subset_table = subset_table.drop(subset_table[subset_table["SURF_COND_DESC"] == "Unknown"].index)
subset_table = subset_table.drop(subset_table[subset_table["SURF_COND_DESC"] == "Not Applicable"].index)
subset_size_reducer = default_rng().choice(subset_table.index, size=116500, replace=False)
subset_table = subset_table.drop(index = subset_size_reducer)

# count for # of pins on the map before displaying it
#count = 0

# Iterate each row in the subset table and mark them on the map as green pins for dry roads and red pins for wet/snowy roads
for i, crash in subset_table[subset_table["YEAR"] == 2020].iterrows():
    if crash["SURF_COND_DESC"] == "Dry":
        folium.Marker(location = [crash["LATITUDE"], crash["LONGITUDE"]], icon = folium.Icon(color = "green")).add_to(map_osm)
        #count+=1
        
    if crash["SURF_COND_DESC"] == "Wet":
        folium.Marker(location = [crash["LATITUDE"], crash["LONGITUDE"]], icon = folium.Icon(color = "red")).add_to(map_osm)
        #count+=1

    if crash["SURF_COND_DESC"] == "Snow":
        folium.Marker(location = [crash["LATITUDE"], crash["LONGITUDE"]], icon = folium.Icon(color = "red")).add_to(map_osm)
        #count+=1

#print(count)
map_osm

I am using the vehicle crash report dataset from the state government website: opendata.maryland.gov. I extracted a datatable from the website and then manipulated it to determine the relationship between the road conditions and the number of crashes. The result map above shows that there are more vehicle crashes on dry roads (green pins) than on wet/snowy roads (red pins) in Baltimore. Altough, only about 31% of days in a year on average in Baltimore are rainy/snowy days, the number gap is huge and so it still does not defeat the observation I have made that there are usually more car crashes on dry roads; my guess is that many drivers are aware of the danger of wet/snowy roads and thus they drive safe, which could be a factor in reducing the number of car crashes.